# New York City Taxi Trip Duration

The competition dataset is based on the 2016 NYC Yellow Cab trip record data made available in Big Query on Google Cloud Platform. The data was originally published by the NYC Taxi and Limousine Commission (TLC). The data was sampled and cleaned for the purposes of this playground competition. Based on individual trip attributes, participants should **predict the duration of each trip in the test set.**

## I. Data loading and overview <a id="one"></a>

In [5]:
import numpy as np
np.random.seed(42)
import pandas as pd

import shap

from sklearn.metrics import mean_squared_log_error as MSE
from sklearn.model_selection import train_test_split

import xgboost as xgb
import lightgbm as lgb


### a. Loading the data <a id="one-a"></a>

In [6]:
df = pd.read_csv("/kaggle/input/nyc-taxi-trip-duration/train.zip")
test = pd.read_csv("/kaggle/input/nyc-taxi-trip-duration/test.zip")

In [7]:
add_on_df = pd.read_csv("/kaggle/input/2014-new-york-city-taxi-trips/nyc_taxi_data_2014.csv")

In [8]:
add_on_df.dropna(inplace=True,)
trip_duration = pd.to_datetime(add_on_df.dropoff_datetime) - pd.to_datetime(add_on_df.pickup_datetime)
trip_duration /= np.timedelta64(1, 's')
add_on_df["trip_duration"] = trip_duration
add_on_df.drop(add_on_df[add_on_df.trip_duration < 0].index, inplace=True)
trip_duration = add_on_df["trip_duration"]

revelant_columns = list(test.columns) 
revelant_columns.remove("id")

add_on_df = add_on_df[revelant_columns]
# add_on_df["vendor_id"] = add_on_df["vendor_id"].astype('category')
# add_on_df["vendor_id"] = add_on_df["vendor_id"].cat.codes
add_on_df.shape, trip_duration.shape

((7362993, 8), (7362993,))

In [9]:
# For Generalisation Hypothesis Experiments, uncomment the following to assign the add-on df to test variable, overwriting the testing test
# test = add_on_df[:250_000]
# trip_duration = trip_duration[:250_000]

In [10]:
test.shape

(625134, 9)

### b. Overview <a id="one-b"></a>

Column | Description
------- | -------
**id** | a unique identifier for each trip  
**vendor_id** | a code indicating the provider associated with the trip record  
**pickup_datetime** | date and time when the meter was engaged  
**dropoff_datetime** | date and time when the meter was disengaged  
**passenger_count** | the number of passengers in the vehicle (driver entered value)  
**pickup_longitude** | the longitude where the meter was engaged  
**pickup_latitude** | the latitude where the meter was engaged  
**dropoff_longitude** | the longitude where the meter was disengaged  
**dropoff_latitude** | the latitude where the meter was disengaged  
**store_and_fwd_flag** | This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server (Y=store and forward; N=not a store and forward trip)  
**trip_duration** | duration of the trip in seconds  

*Disclaimer: The decision was made to not remove dropoff coordinates from the dataset order to provide an expanded set of variables to use in Kernels.*

## II. Data cleaning <a id="two"></a>

### a. Duplicated and missing values <a id="two-a"></a>

In [11]:
#Count the number of duplicated rows
df.duplicated().sum()

0

In [12]:
#Count the number of NaN values for each column
df.isna().sum()

id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64

In [13]:
add_on_df.isna().sum()

vendor_id             0
pickup_datetime       0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
dtype: int64

In [14]:
add_on_df.shape

(7362993, 8)

There are no duplicated or missing values.

### b. Deal with outliers <a id="two-b"></a>

We clearly see `trip_duration` takes strange values for `min` and `max`. Let's have a quick visualization with a boxplot.

There are outliers for `trip_duration`. I can't find a proper interpretation and it will probably damage our model, so I choose to get rid of them.

In [15]:
#Only keep trips that lasted less than 5900 seconds
df = df[(df.trip_duration < 5900)]

#Only keep trips with passengers
df = df[(df.passenger_count > 0)]

## III. Features engineering <a id="three"></a>

### b. Deal with categorical features <a id="three-b"></a>

In [16]:
#One-hot encoding binary categorical features
df = pd.concat([df, pd.get_dummies(df['store_and_fwd_flag'])], axis=1)
test = pd.concat([test, pd.get_dummies(test['store_and_fwd_flag'])], axis=1)

df.drop(['store_and_fwd_flag'], axis=1, inplace=True)
test.drop(['store_and_fwd_flag'], axis=1, inplace=True)


df = pd.concat([df, pd.get_dummies(df['vendor_id'])], axis=1)
test = pd.concat([test, pd.get_dummies(test['vendor_id'])], axis=1)

df.drop(['vendor_id'], axis=1, inplace=True)
test.drop(['vendor_id'], axis=1, inplace=True)

In [17]:
#Datetyping the dates
df['pickup_datetime'] = pd.to_datetime(df.pickup_datetime)
test['pickup_datetime'] = pd.to_datetime(test.pickup_datetime)

df.drop(['dropoff_datetime'], axis=1, inplace=True) #as we don't have this feature in the testset

#Date features creations and deletions
df['month'] = df.pickup_datetime.dt.month
df['week'] = df.pickup_datetime.dt.week
df['weekday'] = df.pickup_datetime.dt.weekday
df['hour'] = df.pickup_datetime.dt.hour
df['minute'] = df.pickup_datetime.dt.minute
df['minute_oftheday'] = df['hour'] * 60 + df['minute']
df.drop(['minute'], axis=1, inplace=True)

test['month'] = test.pickup_datetime.dt.month
test['week'] = test.pickup_datetime.dt.week
test['weekday'] = test.pickup_datetime.dt.weekday
test['hour'] = test.pickup_datetime.dt.hour
test['minute'] = test.pickup_datetime.dt.minute
test['minute_oftheday'] = test['hour'] * 60 + test['minute']
test.drop(['minute'], axis=1, inplace=True)

df.drop(['pickup_datetime'], axis=1, inplace=True)

df.info()

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1455957 entries, 0 to 1458643
Data columns (total 16 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   id                 1455957 non-null  object 
 1   passenger_count    1455957 non-null  int64  
 2   pickup_longitude   1455957 non-null  float64
 3   pickup_latitude    1455957 non-null  float64
 4   dropoff_longitude  1455957 non-null  float64
 5   dropoff_latitude   1455957 non-null  float64
 6   trip_duration      1455957 non-null  int64  
 7   N                  1455957 non-null  uint8  
 8   Y                  1455957 non-null  uint8  
 9   1                  1455957 non-null  uint8  
 10  2                  1455957 non-null  uint8  
 11  month              1455957 non-null  int64  
 12  week               1455957 non-null  int64  
 13  weekday            1455957 non-null  int64  
 14  hour               1455957 non-null  int64  
 15  minute_oftheday    1455957 non-n

### d. Distance and speed creations <a id="three-d"></a>

In [18]:
#Function aiming at calculating distances from coordinates
def ft_haversine_distance(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371 #km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

#Add distance feature
df['distance'] = ft_haversine_distance(df['pickup_latitude'].values,
                                                 df['pickup_longitude'].values, 
                                                 df['dropoff_latitude'].values,
                                                 df['dropoff_longitude'].values)
test['distance'] = ft_haversine_distance(test['pickup_latitude'].values, 
                                                test['pickup_longitude'].values, 
                                                test['dropoff_latitude'].values, 
                                                test['dropoff_longitude'].values)

In [19]:
#Function aiming at calculating the direction
def ft_degree(lat1, lng1, lat2, lng2):
    AVG_EARTH_RADIUS = 6371 #km
    lng_delta_rad = np.radians(lng2 - lng1)
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))

#Add direction feature
df['direction'] = ft_degree(df['pickup_latitude'].values,
                                df['pickup_longitude'].values,
                                df['dropoff_latitude'].values,
                                df['dropoff_longitude'].values)
test['direction'] = ft_degree(test['pickup_latitude'].values,
                                  test['pickup_longitude'].values, 
                                  test['dropoff_latitude'].values,
                                  test['dropoff_longitude'].values)

In [20]:
#Remove distance outliers
df = df[(df.distance < 200)]

#Create speed feature
df['speed'] = df.distance / df.trip_duration

#Remove speed outliers
df = df[(df.speed < 30)]
df.drop(['speed'], axis=1, inplace=True)

## IV. Model selection <a id="four"></a>

### a. Split <a id="four-a"></a>

the 250_000 train_size 250_000 test size

In [21]:
#Split the labeled data frame into two sets: features and target
y = df["trip_duration"]
df.drop(["trip_duration"], axis=1, inplace=True)
df.drop(['id'], axis=1, inplace=True)
X = df

X.shape, y.shape

((1455946, 16), (1455946,))

In [22]:
#Split the labeled data frame into two sets to train then test the models

X_train, X_rest, y_train, y_rest = train_test_split(X, y, train_size=250_000, random_state=42)

X_test, X_rest, y_test, y_rest = train_test_split(X_rest, y_rest, train_size=250_000, random_state=42)

X_train.shape, y_train.shape, 

((250000, 16), (250000,))

In [ ]:
def add_on_df_clean_up():
    test.drop('pickup_datetime', axis=1, inplace=True)
    test.insert(8,2,0)
    test.rename(columns={"CMT": 1},inplace=True)
    X_add_on, _, y_add_on, _ = train_test_split(test, trip_duration, train_size=75_000, random_state=42)
    return X_add_on, y_add_on

In [ ]:
# Uncomment the following for Generalisation Hypothesis tests
# X_add_on, y_add_on = add_on_df_clean_up()

### b. Metrics <a id="four-b"></a>
For this specific problem, we'll measure the error using the RMSLE (Root Mean Squared Log Error).

### c. Models <a id="four-c"></a>

#### Light GBM Models

**model 1:**  


**model 2:**  

In [ ]:
%%time

lgb_params = {
    'metric': 'rmse',
    'is_training_metric': True,
    'learning_rate': 0.1,
    'max_depth': 25,
    'num_leaves': 1000, 
    'objective': 'regression',
    'feature_fraction': 0.9,
    'bagging_fraction': 0.5,
    'max_bin': 1000 ,
}

lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test)
lgbm1 = lgb.train(lgb_params, lgb_train, num_boost_round=100, valid_sets=[lgb_train], early_stopping_rounds=5,verbose_eval = -1)

In [ ]:
%%time
from lightgbm import LGBMRegressor

lgbm2 = lgb.LGBMRegressor(n_estimators=500, num_leaves=1000, max_depth=25, objective='regression') #0.39507740906150585
# lgbm = lgb.LGBMRegressor() #0.3990924865808348
lgbm2.fit(X_train, y_train)

#### XGB

In [ ]:
xgb_params = {
    'booster':            'gbtree',
    'objective':          'reg:squarederror',
    'learning_rate':      0.1,
    'max_depth':          14,
    'subsample':          0.8,
    'colsample_bytree':   0.7,
    'colsample_bylevel':  0.7,
    'silent':             1
}

In [ ]:
%%time
xgbm = xgb.XGBRegressor(**xgb_params,num_boost_round = 200)
xgbm.fit(X_train, y_train)

In [23]:
%%time
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)
params = {'max_depth':15, #maximum depth of a tree 8 12
         'objective':'reg:linear',
         'eta'      :0.05, #0.3
         'subsample':0.9,#SGD will use this percentage of data 0.8 0.99
         'lambda'  :3, #L2 regularization term,>1 more conservative 4 
         'colsample_bytree':0.6, #0.9
         'colsample_bylevel':0.7, #1 0.7
         'min_child_weight': 0.5, #10 0.5
         #'nthread'  :3 ... default is max cores
         'eval_metric':'rmse'}  #number of cpu core to use
# running for 2k iterations 
xgbm2 = xgb.train(params, dtrain, num_boost_round=125, maximize=False, verbose_eval=100)
print(np.sqrt(MSE(y_test, np.clip(xgbm2.predict(dtest),0,None))), end="\n\n")

[17:32:31] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
0.38125934818136115

CPU times: user 2min 22s, sys: 2.51 s, total: 2min 24s
Wall time: 1min 12s


In [ ]:
models = [lgbm1,lgbm2,xgbm]

In [ ]:
for model in models:
    print(str(model))
    print(np.sqrt(MSE(y_train, np.clip(model.predict(X_train),0,None))), end="\n\n")

In [ ]:
# d_add_test = xgb.DMatrix(test)
print(np.sqrt(MSE(y_train, np.clip(xgbm2.predict(dtrain),0,None))), end="\n\n")

models.append(xgbm2)
models

## Explanations

In [ ]:
_, exp_data, _, y_exp_data = train_test_split(X, y, test_size=10000, random_state=42)

#### Shap values

In [ ]:
def get_shap_vals(model):
    exp = shap.Explainer(model,X_train)
    shap_vals = exp.shap_values(exp_data,check_additivity=False)
    return shap_vals

In [ ]:
shap_vals = np.empty((len(models), *exp_data.shape))
for i,model in enumerate(models):
    print(i)
    shap_vals[i] = get_shap_vals(model)

In [ ]:
shap_vals.shape

#### similarity

In [ ]:
def shap_similarity(epsilon=20, normalize=True, measure="T"):
    sim_matrix = np.zeros([len(models),len(models)])
    for i in range(len(models)-1):
        for j in range(i+1,len(models)):
            shap_vals1 = shap_vals[i]
            shap_vals2 = shap_vals[j]
            if measure == "T":
                similarity = np.count_nonzero(np.abs(shap_vals1-shap_vals2) < epsilon) 
                if normalize:
                    similarity /= np.prod(shap_vals1.shape) 
            elif measure == "S_delta":
                similarity = np.count_nonzero(np.sum(np.abs(shap_vals1-shap_vals2),axis=1)<epsilon*shap_vals1.shape[1])
                if normalize:
                    similarity /= shap_vals1.shape[0]
            elif measure == "S":
                similarity = np.count_nonzero(np.max(np.abs(shap_vals1-shap_vals2),axis=1)<epsilon)
                if normalize:
                    similarity /= shap_vals1.shape[0]
            print(str(models[i])[:120],"vs",str(models[j])[:120],similarity, sep="\n", end="\n\n")
            sim_matrix[i,j] = sim_matrix[j, i] = similarity    
    return np.sum(sim_matrix,axis=0) / (len(models)-1)

In [ ]:
shap_similarity()

In [ ]:
delta = [10, 12.5, 15, 17.5, 20]

In [ ]:
#T-sim 
T_delta_20 =  [20, 0.7546    , 0.69835   , 0.76955833, 0.7540625]
T_delta_175 = [17.5, 0.72081458, 0.6654    , 0.73611042, 0.7203  ]
T_delta_15 =  [15, 0.68227292, 0.62793125, 0.69672292, 0.68137292]
T_delta_125 = [12.5, 0.6366875 , 0.58493958, 0.65024583, 0.63460625]
T_delta_10 =  [10, 0.58339583, 0.53530833, 0.59498125, 0.58037708]
rows = [T_delta_20 ,
        T_delta_175,
        T_delta_15 ,
        T_delta_125,
        T_delta_10 ,]
cols = ["delta", "lgbm1", "lgbm2" ,"xgbm1","xgbm2"]
T_delta_df = pd.DataFrame(rows,columns=cols)
T_delta_df

In [ ]:
import seaborn as sns
sns.set(rc = {'figure.figsize':(15,8)})
sns.set_theme(style="whitegrid")
    

In [ ]:
sim_fig = sns.lineplot(x="delta", y="value", hue="variable", style ="variable", markers=True, data=pd.melt(T_delta_df,["delta"]))
sim_fig.set_ylabel("average similarity coefficient")
sim_fig.set_yticks(np.linspace(0.5,0.8,11))

In [ ]:
#S-simlarity 
delta_10 =  [10,0.24066667, 0.07083333, 0.3572    , 0.3327   ]
delta_125 = [12.5, 0.46983333, 0.21253333, 0.53946667, 0.4827 ]
delta_15 =  [15, 0.64136667, 0.40136667, 0.67413333, 0.59986667]
delta_175 = [17.5, 0.75196667, 0.579     , 0.77223333, 0.70686667]
delta_20 =  [20, 0.81503333, 0.69933333, 0.83596667, 0.792 ]

rows = [delta_20 ,
        delta_175,
        delta_15 ,
        delta_125,
        delta_10 ,]
cols = ["delta", "lgbm1", "lgbm2" ,"xgbm1","xgbm2"]
S_delta_df = pd.DataFrame(rows,columns=cols)

In [ ]:
sim_fig = sns.lineplot(x="delta", y="value", hue="variable", style ="variable", markers=True, data=pd.melt(S_delta_df,["delta"]))
sim_fig.set_ylabel("average similarity coefficient")

####  conflict

In [ ]:
def shap_conflict(alpha=400,normalize=True,measure="F"):
    conf_matrix = np.zeros([len(models),len(models)])
    for i in range(len(models)-1):
        for j in range(i+1,len(models)):
            shap_vals1 = shap_vals[i]
            shap_vals2 = shap_vals[j]
            conf_cond = shap_vals1*shap_vals2<0
            dist_cond = np.abs(shap_vals1-shap_vals2)> alpha
            satifies_conditions = np.logical_and(dist_cond,conf_cond)
            print(np.nonzero(satifies_conditions))
            if measure == "F":
                conflict = np.sum(satifies_conditions)
                if normalize:
                    conflict /= np.prod(shap_vals1.shape)
            elif measure == "C":
                conflict = np.count_nonzero(np.max(satifies_conditions, axis=1))
                if normalize:
                    conflict /= shap_vals1.shape[0]
            print(str(models[i])[:120],"vs",str(models[j])[:120],conflict, sep="\n", end="\n\n")
            conf_matrix[i,j] = conf_matrix[j, i] = conflict    
    return np.sum(conf_matrix,axis=0) / (len(models)-1)

In [ ]:
shap_conflict()

##### Conflict examples

In [ ]:
shap_vals[0][5725,3]

In [ ]:
shap_vals[1][5725,3]

In [ ]:
data_index = 1184

In [ ]:
shap.bar_plot(shap_vals[0][data_index],X.iloc[[data_index]])

In [ ]:
shap.bar_plot(shap_vals[1][data_index],X.iloc[[data_index]])

In [ ]:
shap.summary_plot(shap_vals[0], exp_data)

##### Conflict scores line plots 

In [ ]:
alpha = [80,100,120,140,160]

In [ ]:
#C-conf
alpha_80 = np.array([80,0.06383333, 0.1183    , 0.0439    , 0.05123333])
alpha_100 =np.array([100,0.04486667, 0.0859    , 0.02773333, 0.0335    ])
alpha_120= np.array([120,0.03376667, 0.06806667, 0.01966667, 0.02516667])
alpha_140 =np.array([140,0.0262    , 0.05486667, 0.0146    , 0.0196    ])
alpha_160 =np.array([160,0.0219    , 0.04466667, 0.0111    , 0.01473333])


rows = [alpha_80 ,
        alpha_100,
        alpha_120,
        alpha_140,
        alpha_160,]
cols = ["alpha", "lgbm1", "lgbm2" ,"xgbm1","xgbm2"]
alpha_df = pd.DataFrame(rows,columns=cols)

In [ ]:
C_conf_fig = sns.lineplot(x="alpha", y="value", hue="variable", style ="variable", markers=True, data=pd.melt(alpha_df,["alpha"]))
C_conf_fig.set_ylabel("average conflict coefficient")
C_conf_fig.set_yticks(np.linspace(0.01,0.12,12))

In [ ]:
#F-conf
F_alpha_80  = np.array([80,0.00433333, 0.00865208, 0.00318542, 0.00385417])
F_alpha_100 = np.array([100,0.00297292, 0.00611875, 0.00195625, 0.00248958])
F_alpha_120 = np.array([120,0.00222708, 0.0047125 , 0.00135833, 0.00180625])
F_alpha_140 = np.array([140,0.00172292, 0.00374167, 0.00098333, 0.00138542])
F_alpha_160 = np.array([160,0.00143542, 0.00300833, 0.00074167, 0.00102292])

rows = [F_alpha_80 ,
        F_alpha_100,
        F_alpha_120,
        F_alpha_140,
        F_alpha_160,]
cols = ["alpha", "lgbm1", "lgbm2" ,"xgbm1","xgbm2"]
F_alpha_df = pd.DataFrame(rows,columns=cols)

In [ ]:
F_conf_fig = sns.lineplot(x="alpha", y="value", hue="variable", style ="variable", markers=True, data=pd.melt(F_alpha_df,["alpha"]))
F_conf_fig.set_ylabel("average conflict coefficient")

### SRA results

In [ ]:
def top_features(datapoint_shap_vals, nfeatures=7):
    """gives the indexes of the n features with the biggest absolute shap vals"""
    return np.argpartition(abs(datapoint_shap_vals), -nfeatures)[-nfeatures:]
    

def equal_ranking(feat, top_feats1, top_feats2):
    return list(np.where(top_feats1==feat)[0]) == list(np.where(top_feats2==feat)[0])

def aggregated_sra(k=3, agg=np.mean):
    sra_matrix = np.zeros([len(all_shap_vals),len(all_shap_vals)])
    for i in range(len(all_shap_vals)-1):
        for j in range(i+1,len(all_shap_vals)):
            shap_vals1 = all_shap_vals[i]
            shap_vals2 = all_shap_vals[j]
            ndatapoints = len(shap_vals1)
            model_sras = np.empty([ndatapoints])
            for l in range(ndatapoints):
                sra = 0
                top_feats1 = top_features(shap_vals1[l], nfeatures=k)
                top_feats2 = top_features(shap_vals2[l], nfeatures=k)
                for feat in top_feats1:
                    if equal_ranking(feat, top_feats1, top_feats2) and shap_vals1[l][feat]*shap_vals2[l][feat]>=0:
                        sra += 1
                sra /= k
                model_sras[l] = sra
            sra_matrix[i, j] = sra_matrix[j,i] =agg(model_sras)
    print(sra_matrix)
    return np.sum(sra_matrix,axis=0) / (len(all_shap_vals)-1)


In [ ]:
aggregated_sra()